In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import create_model
import inspect, json
from inspect import Parameter
from groq import Groq
print(load_dotenv())

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

True


## **Custom agent**

**Define the functions**

In [2]:
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [3]:
abc(2, 3)

-2

In [4]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [5]:
abc_json = jsonschema(abc)
abc_json

C:\Users\Arushi Saxena\AppData\Local\Temp\ipykernel_21232\3911407637.py:13: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'abc',
 'description': 'Compute abc between two numbers',
 'parameters': {'properties': {'num1': {'title': 'Num1', 'type': 'integer'},
   'num2': {'title': 'Num2', 'type': 'integer'}},
  'required': ['num1', 'num2'],
  'title': 'Input for `abc`',
  'type': 'object'}}

In [6]:
model_name = "llama3-8b-8192"

**Ask GPT**

In [7]:
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "compute abc between 2 and 3"},
    ]
)

print(response.choices[0].message.content)

To compute the arithmetic progression (AP) sum abc, I'll need some information from you:

* The first term (a): 
* The second term (b): 
* The common difference (c): 

You mentioned the range is between 2 and 3. Is that for the common difference (c)? 

If so, let's calculate the sum abc:

1. The first term (a): Please provide a value between 2 and 3. For example, let's say a = 2.1.
2. The second term (b): Since c is between 2 and 3, we can assume c = 2.5. Now, we can calculate b = a + c = 2.1 + 2.5 = 4.6.
3. The sum abc: The formula for the sum of an arithmetic series is:

S = n/2 * (2a + (n-1)d)

In this case, since you didn't specify the number of terms (n), let's assume you want to calculate the sum up to the fifth term (n = 5). 

S = 5/2 * (2*4.6 + (5-1)*2.5) = 5/2 * (9.2 + 10) = 5/2 * 19.2 = 48

The arithmetic progression sum abc from 2.1 to 4.6 with a common difference of 2.5 up to five terms is 48. 

Please provide the correct values for a and n if you'd like a different calcula

In [8]:
response.choices[0].message.content

"To compute the arithmetic progression (AP) sum abc, I'll need some information from you:\n\n* The first term (a): \n* The second term (b): \n* The common difference (c): \n\nYou mentioned the range is between 2 and 3. Is that for the common difference (c)? \n\nIf so, let's calculate the sum abc:\n\n1. The first term (a): Please provide a value between 2 and 3. For example, let's say a = 2.1.\n2. The second term (b): Since c is between 2 and 3, we can assume c = 2.5. Now, we can calculate b = a + c = 2.1 + 2.5 = 4.6.\n3. The sum abc: The formula for the sum of an arithmetic series is:\n\nS = n/2 * (2a + (n-1)d)\n\nIn this case, since you didn't specify the number of terms (n), let's assume you want to calculate the sum up to the fifth term (n = 5). \n\nS = 5/2 * (2*4.6 + (5-1)*2.5) = 5/2 * (9.2 + 10) = 5/2 * 19.2 = 48\n\nThe arithmetic progression sum abc from 2.1 to 4.6 with a common difference of 2.5 up to five terms is 48. \n\nPlease provide the correct values for a and n if you'd l

In [9]:
messages= [
    {"role": "user", "content": "Compute abc between 2 and 3"}
]

# Pass th function to GPT model
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    functions=[abc_json],
    function_call="auto",
    temperature=0
)

In [10]:
response

ChatCompletion(id='chatcmpl-2039f020-8564-4b66-859b-caaf0cd7aa27', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=FunctionCall(arguments='{"num1":2,"num2":3}', name='abc'), reasoning=None, tool_calls=None))], created=1752298839, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_24ec19897b', usage=CompletionUsage(completion_tokens=70, prompt_tokens=896, total_tokens=966, completion_time=0.077808246, prompt_time=0.098478039, queue_time=0.045089879999999985, total_time=0.176286285), usage_breakdown=None, x_groq={'id': 'req_01jzyjcq1cen8ryxk581evj2sx'}, service_tier='on_demand')

**Executing the function by extracting the info from the output of the model**

In [11]:
print(response.choices[0].message.function_call)
print(response.choices[0].message.function_call.arguments)
print(type(response.choices[0].message.function_call.arguments))

FunctionCall(arguments='{"num1":2,"num2":3}', name='abc')
{"num1":2,"num2":3}
<class 'str'>


In [12]:
func_name = response.choices[0].message.function_call.name
func_args = json.loads(response.choices[0].message.function_call.arguments)
print("Function name:", func_name)
print("Function arguments:", func_args)
print(type(func_args))

Function name: abc
Function arguments: {'num1': 2, 'num2': 3}
<class 'dict'>


In [13]:
if func_name == 'abc':
    result = abc(**func_args)
print(result)

-2


## **Using Langchain**

In [14]:
from langchain_core.tools import tool

@tool
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [15]:
abc.description

'Compute abc between two numbers'

In [16]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama3-8b-8192",
    groq_api_key=os.getenv("GROQ_API_KEY")
)


In [17]:
tools = [abc]

llm_with_tools = llm.bind_tools(tools)

In [18]:
response = llm_with_tools.invoke("Compute abc between 2 and 3")

In [19]:
response.additional_kwargs

{'tool_calls': [{'id': 'tmexy153q',
   'function': {'arguments': '{"num1":2,"num2":3}', 'name': 'abc'},
   'type': 'function'}]}